In [1]:

from enum import unique
import streamlit as st 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import plotly.express as px
import plotly.graph_objects as go
import time # to simulate a real time data, time loop
st.set_page_config(page_title='Live Machine Performance Dashboard',layout='wide')
a= st.number_input("SKU", min_value=100, max_value=125, value= 125)
if a==100:
    SKU= 90;
if a== 125:
    SKU= 84;
    
option = st.selectbox('MACHINE LEVEL',('STAMPER', 'BOSCH1','BOSCH2', 'CPM1','CPM2'))

##SKU= st.sidebar.selectbox('SKU', ('Wheel 125g', 'Wheel 100g'))
from datetime import datetime



placeholder_container1 = st.empty()


for seconds in range(200):
#while True: 

    with placeholder_container1.container():
        df_stamper= pd.read_csv("L4Stamper.csv")
        df_stamper_without_dash = df_stamper.iloc[1:]
               
        df_CPM1= pd.read_csv("L4CPM1.csv")
        df_CPM1_without_dash = df_CPM1.iloc[1:]
        
        df_CPM2= pd.read_csv("L4CPM2.csv")
        df_CPM2_without_dash = df_CPM2.iloc[1:]

        df_BOSCH1= pd.read_csv("L4BOSCH1.csv")
        df_BOSCH1_without_dash = df_BOSCH1.iloc[1:]
        
        df_BOSCH2= pd.read_csv("L4BOSCH2.csv")
        df_BOSCH2_without_dash = df_BOSCH2.iloc[1:]
      

        # create two columns for charts 

        fig_col1, fig_col2, fig_col3 = st.columns([0.75,2,2],gap="small")
        
        
        
        
        with fig_col1:
            
###################################            # FUNCTION FOR LINE OEE
            def line_oee(oee):
                lizard_007 = go.Figure(go.Indicator(
                domain = {'x': [0, 1], 'y': [0, 1]},
                value = oee*100, #put line OEE here. 
                mode = "gauge+number+delta",
                title = {'text': "LINE OEE"},
                delta = {'reference': 70},
                gauge = {'axis': {'range': [0, 100]},

                         'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 70}}))
                lizard_007.update_layout(
                autosize=False,
               
                width=320,
                height=320,
                margin_l=0
            )

                st.write(lizard_007)
            line_csv = pd.read_csv("Line.csv")
            line_csv_without_dash = line_csv.iloc[1:,:]
            dt3 = datetime.strptime('2021-10-20 06:00:48.00000', "%Y-%m-%d %H:%M:%S.%f")
            dt_line =line_csv_without_dash._get_value(1,0, takeable = True)
            line_time = datetime.strptime(dt_line, "%Y-%m-%d %H:%M:%S.%f")
            
            line_time_diff=round(((line_time-dt3).seconds)/60)
            if line_time_diff<480:
                fbc= line_csv_without_dash._get_value(1,3, takeable = True)
                fbc = float(fbc)
                oee=(fbc*SKU)/(440*line_time_diff)
                line_oee(oee)

            if line_time_diff>480 and line_time_diff<960:
                fbc= line_csv_without_dash._get_value(1,3, takeable = True)
                fbc = float(fbc)
                oee=(fbc*SKU)/(440*(line_time_diff-480))
                line_oee(oee)



            if line_time_diff>960:
                fbc= line_csv_without_dash._get_value(1,3, takeable = True)
                fbc = float(fbc)
                oee=(fbc*SKU)/(440*(line_time_diff-960))
                line_oee(oee)
            

            
            
            
            
            
#################################################################################################################################################################################################################################################
            #fig_bar = go.Figure([go.Bar(x=oee, y=[20, 14, 23])])

            
        with fig_col2:
           
                    
###### FUNCTION FOR BREAKDOWN TIME
                def breakdown_time_and_minor_stop_time(file_name,run_col):
                    
                        line_csv = pd.read_csv(file_name)
                        line_csv = line_csv.fillna(0)
                        line_csv_without_dash = line_csv.iloc[1:,:]
                        dt3 = datetime.strptime('2021-10-20 06:00:48.00000', "%Y-%m-%d %H:%M:%S.%f")
                        dt_line =line_csv_without_dash._get_value(1,0, takeable = True)
                        line_time = datetime.strptime(dt_line, "%Y-%m-%d %H:%M:%S.%f")

                        line_time_diff=round(((line_time-dt3).seconds)/60)
                        k = line_time_diff
                        k= int(k)

                        if k<480:
                            count =0
                            minor_stop = 0
                            breakdown = 0
                            minor_stop_time = 0
                            breakdown_time =0
                            for total in range(0,k):
                                if float(line_csv_without_dash._get_value(total+1,run_col, takeable = True))==0.0:
                                    count=  count+1


                                if float(line_csv_without_dash._get_value(total+1,run_col, takeable = True))==1.0:
                                    if count < 10 and count>0:
                                        minor_stop = minor_stop +1
                                        minor_stop_time = minor_stop_time + count

                                        count = 0
                                    if count > 10:
                                        breakdown = breakdown+1
                                        breakdown_time = breakdown_time + count

                                        count= 0
                                if (k-total)==1:
                                    if count < 10 and count>0:
                                        minor_stop = minor_stop +1
                                        minor_stop_time = minor_stop_time + count
                                        count = 0
                                    if count > 10:
                                        breakdown = breakdown+1
                                        breakdown_time = breakdown_time + count

                                        count= 0




                        if k>480 and k<960:
                            count =0
                            minor_stop = 0
                            breakdown = 0
                            minor_stop_time = 0
                            breakdown_time =0
                            for total in range(0,k-480):
                                if float(line_csv_without_dash._get_value(total+1,run_col, takeable = True))==0.0:
                                    count=  count+1


                                if float(line_csv_without_dash._get_value(total+1,run_col, takeable = True))==1.0:
                                    if count < 10 and count>0:
                                        minor_stop = minor_stop +1
                                        minor_stop_time = minor_stop_time + count
                                        count = 0
                                    if count > 10:
                                        breakdown = breakdown+1
                                        breakdown_time = breakdown_time + count

                                        count= 0
                                if (k-total)==481:
                                    if count < 10 and count>0:
                                        minor_stop = minor_stop +1
                                        minor_stop_time = minor_stop_time + count
                                        count = 0
                                    if count > 10:
                                        breakdown = breakdown+1
                                        breakdown_time = breakdown_time + count

                                        count= 0



                        if k>960:

                            count =0
                            minor_stop = 0
                            breakdown = 0
                            minor_stop_time = 0
                            breakdown_time =0
                            for total in range(0,k-960):
                                if float(line_csv_without_dash._get_value(total+1,run_col, takeable = True))==0.0:
                                    count=  count+1



                                if float(line_csv_without_dash._get_value(total+1,run_col, takeable = True))==1.0:
                                    if count < 10 and count>0:
                                        minor_stop = minor_stop +1
                                        minor_stop_time = minor_stop_time + count
                                        count = 0

                                    if count > 10:
                                        breakdown = breakdown+1
                                        breakdown_time = breakdown_time + count

                                        count= 0
                                        
                                if (k-total)==961:
                                    if count < 10 and count>0:
                                        minor_stop = minor_stop +1
                                        minor_stop_time = minor_stop_time + count
                                        count = 0
                                    if count > 10:
                                        breakdown = breakdown+1
                                        breakdown_time = breakdown_time + count

                                        count= 0


                        return(breakdown_time,minor_stop_time)
                    
                    
                    
############ fUNCCTION fOR sPEED lOSS                        
                def speed_loss_and_oee(file_name,output_count1,speed):
                        line_csv = pd.read_csv(file_name)
                        line_csv.replace(np.nan,0)
                        line_csv_without_dash = line_csv.iloc[1:,:]
                        dt3 = datetime.strptime('2021-10-20 06:00:48.00000', "%Y-%m-%d %H:%M:%S.%f")
                        dt_line =line_csv_without_dash._get_value(1,0, takeable = True)
                        line_time = datetime.strptime(dt_line, "%Y-%m-%d %H:%M:%S.%f")

                        line_time_diff=round(((line_time-dt3).seconds)/60)
                        output_count = float(line_csv_without_dash._get_value(1,output_count1, takeable = True))
                        print(output_count)
                        if line_time_diff<480:
                            present_time = (line_time_diff - 0)
                            OEE = output_count/500
                            speed_loss = ((500*present_time)-output_count)/500
                        if line_time_diff>480 and line_time_diff<960:
                            present_time = line_time_diff - 480
                            OEE = output_count/500
                            speed_loss = ((500*present_time)-output_count)/500


                        if line_time_diff>960:
                            present_time = line_time_diff - 960
                            OEE = output_count/500
                            speed_loss = ((500*present_time)-output_count)/500

                        return(OEE, speed_loss)
                    
                    
                def speed_loss_and_oee_2(file_name,output_count1,speed):
                        line_csv = pd.read_csv(file_name)
                        line_csv.replace(np.nan,0)
                        line_csv_without_dash = line_csv.iloc[1:,:]
                        dt3 = datetime.strptime('2021-10-20 06:00:48.00000', "%Y-%m-%d %H:%M:%S.%f")
                        dt_line =line_csv_without_dash._get_value(1,0, takeable = True)
                        line_time = datetime.strptime(dt_line, "%Y-%m-%d %H:%M:%S.%f")

                        line_time_diff=round(((line_time-dt3).seconds)/60)
                        output_count = float(line_csv_without_dash._get_value(1,output_count1, takeable = True))
                        print(output_count)
                        if line_time_diff<480:
                            present_time = (line_time_diff - 0)
                            OEE2 = (output_count*SKU)/220
                            speed_loss2 = ((220*present_time)-(output_count*SKU))/220
                        if line_time_diff>480 and line_time_diff<960:
                            present_time = line_time_diff - 480
                            OEE2 = (output_count*SKU)/220
                            speed_loss2 = ((220*present_time)-(output_count*SKU))/220


                        if line_time_diff>960:
                            present_time = line_time_diff - 960
                            OEE2 = (output_count*SKU)/220
                            speed_loss2 = ((220*present_time)-(output_count*SKU))/220

                        return(OEE2, speed_loss2)
        
                    
                def speed_loss_and_oee_3(file_name,output_count1,speed):
                        line_csv = pd.read_csv(file_name)
                        line_csv.replace(np.nan,0)
                        line_csv_without_dash = line_csv.iloc[1:,:]
                        dt3 = datetime.strptime('2021-10-20 06:00:48.00000', "%Y-%m-%d %H:%M:%S.%f")
                        dt_line =line_csv_without_dash._get_value(1,0, takeable = True)
                        line_time = datetime.strptime(dt_line, "%Y-%m-%d %H:%M:%S.%f")

                        line_time_diff=round(((line_time-dt3).seconds)/60)
                        output_count = float(line_csv_without_dash._get_value(1,output_count1, takeable = True))
                        print(output_count)
                        if line_time_diff<480:
                            present_time = (line_time_diff - 0)
                            OEE = output_count/250
                            speed_loss = ((250*present_time)-output_count)/250
                        if line_time_diff>480 and line_time_diff<960:
                            present_time = line_time_diff - 480
                            OEE = output_count/250
                            speed_loss = ((250*present_time)-output_count)/250

                        if line_time_diff>960:
                            present_time = line_time_diff - 960
                            OEE = output_count/250
                            speed_loss = ((250*present_time)-output_count)/250

                        return(OEE, speed_loss)  
                    
                    
                if option =="STAMPER":
                    
                    
                    OEE, speed_loss = speed_loss_and_oee("L4STAMPER.csv",10,500)
                    breakdown_time, minor_stop_time = breakdown_time_and_minor_stop_time("L4Stamper.csv",7)
                    speed_loss_minus_stop = speed_loss - minor_stop_time -  breakdown_time
                    
                    st.write("Stamper Loss Capturing")
                    labels = ['Breakdown Time', 'Minor Stop Time',  'OEE', 'Speed Loss/Recycle']
                    values2 = [breakdown_time, minor_stop_time, OEE, speed_loss_minus_stop]
                    fig2 = px.pie(labels, values = values2, names = labels,width=400, height=400)
                    fig2.update_layout(legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1))
                    st.write(fig2)

    #################################################################   MACHINE lOSS CAPTURING            # 
                    
                if option == "CPM1":
                    OEE, speed_loss = speed_loss_and_oee_2("L4CPM1.csv",5,250)
                    breakdown_time, minor_stop_time = breakdown_time_and_minor_stop_time("L4CPM1.csv",3)
                    speed_loss_minus_stop = speed_loss - minor_stop_time -  breakdown_time
                    st.write("CPM1 Loss Capturing")
                    labels = ['Breakdown Time', 'Minor Stop Time',  'OEE', 'Speed Loss']
                    values2 = [breakdown_time, minor_stop_time, OEE, speed_loss_minus_stop]
                    fig2 = px.pie(labels, values = values2, names = labels,width=400, height=400)
                    fig2.update_layout(legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1))
                    st.write(fig2)

#################################################################   MACHINE lOSS CAPTURING            # 
                    
                if option == "CPM2":
                    OEE, speed_loss = speed_loss_and_oee_2("L4CPM2.csv",11,250)
                    breakdown_time, minor_stop_time = breakdown_time_and_minor_stop_time("L4CPM2.csv",3)
                    speed_loss_minus_stop = speed_loss - minor_stop_time -  breakdown_time
                    st.write("CPM2 Loss Capturing")
                    labels = ['Breakdown Time', 'Minor Stop Time',  'OEE', 'Speed Loss']
                    values2 = [breakdown_time, minor_stop_time, OEE, speed_loss_minus_stop]
                    fig2 = px.pie(labels, values = values2, names = labels,width=400, height=400)
                    fig2.update_layout(legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1))
                    st.write(fig2)



    #################################################################   MACHINE lOSS CAPTURING            # 
                    
                if option == "BOSCH1":
                    OEE, speed_loss = speed_loss_and_oee_3("L4BOSCH1.csv",8,250)
                    breakdown_time, minor_stop_time = breakdown_time_and_minor_stop_time("L4BOSCH1.csv",3)
                    speed_loss_minus_stop = speed_loss - minor_stop_time -  breakdown_time
                    st.write("BOSCH1 Loss Capturing")
                    labels = ['Breakdown Time', 'Minor Stop Time',  'OEE', 'Speed Loss']
                    values2 = [breakdown_time, minor_stop_time, OEE, speed_loss_minus_stop]
                    fig2 = px.pie(labels, values = values2, names = labels,width=400, height=400)
                    fig2.update_layout(legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1))
                    st.write(fig2)



    #################################################################   MACHINE lOSS CAPTURING            # 
                    
                if option == "BOSCH2":
                    OEE, speed_loss = speed_loss_and_oee_3("L4BOSCH2.csv",9,250)
                    breakdown_time, minor_stop_time = breakdown_time_and_minor_stop_time("L4BOSCH2.csv",3)
                    speed_loss_minus_stop = speed_loss - minor_stop_time -  breakdown_time
                    st.write("BOSCH2 Loss Capturing")
                    labels = ['Breakdown Time', 'Minor Stop Time',  'OEE', 'Speed Loss']
                    values2 = [breakdown_time, minor_stop_time, OEE, speed_loss_minus_stop]
                    fig2 = px.pie(labels, values = values2, names = labels,width=400, height=400)
                    fig2.update_layout(legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1))
                    st.write(fig2)
#################################################################   MACHINE lOSS CAPTURING         

########################################################################################################################################                #This is the Line Chart  MACHINE SPEED
                
                
                if option == "STAMPER":
                    figl = go.Figure(data=go.Scatter(x=df_stamper_without_dash['DateAndTime            '],y=pd.to_numeric(df_stamper_without_dash["Stamper_Present_Speed   "].iloc[1:481])))
                    figl.update_layout(
                    title="Stamper Running Speed")
                    st.write(figl)
       
    
    
    
                if option == "BOSCH1":
    
                    figl_BOSCH1_speed = go.Figure(data=go.Scatter(x=df_BOSCH1_without_dash['DateAndTime            '],y=pd.to_numeric(df_BOSCH1_without_dash["Bosch_01_Actual_speed   "].iloc[1:481])))
                    figl_BOSCH1_speed.update_layout(
                    title="BOSCH1 Running Speed")
                    st.write(figl_BOSCH1_speed)
                if option == "BOSCH2":
                    
                    
                    figl_BOSCH2_speed = go.Figure(data=go.Scatter(x=df_BOSCH1_without_dash['DateAndTime            '],y=pd.to_numeric(df_BOSCH2_without_dash["BOSCH_2_Speed           "].iloc[1:481])))
                    figl_BOSCH2_speed.update_layout(
                    title="BOSCH2 Running Speed")
                     
                    st.write(figl_BOSCH2_speed)
                    
                    
                    
                if option == "CPM1":
                    
                    figl_CPM1_speed =  go.Figure(data=go.Scatter(x=df_CPM1_without_dash['DateAndTime            '],y=pd.to_numeric(df_CPM1_without_dash["CPM_1_Actual_Speed      "].iloc[1:481])))
                    figl_CPM1_speed.update_layout(
                    title="CPM1 Running Speed")
                    st.write(figl_CPM1_speed)
                if option == "CPM2":
                    
                    
                    figl_CPM2_speed = go.Figure(data=go.Scatter(x=df_CPM2_without_dash['DateAndTime            '],y=pd.to_numeric(df_CPM2_without_dash["CPM_2_Actual_Speed      "].iloc[1:481])))
                    figl_CPM2_speed.update_layout(
                    title="CPM2 Running Speed")
                     
                    st.write(figl_CPM2_speed)
        
###########################################################################################################################################
    # This is the machine Running Status
                if option =="STAMPER":
            
                        
                    df_stamper_without_dash_for_machine_status = df_stamper.iloc[1:480,:]
                    stamper_runmode = pd.to_numeric(df_stamper_without_dash_for_machine_status["Stamper_In_Run_mode     "])
                    stamper_runmode.replace(to_replace={1:0, 0:1}, inplace= True)


                    fig_stamper_runmode = go.Figure()

                    fig_stamper_runmode.add_trace(go.Bar(
                        x=df_stamper_without_dash_for_machine_status['DateAndTime            '],
                        y=stamper_runmode,
                        name='Primary Product',
                        marker_color='red',

                        ))
                    fig_stamper_runmode.update_layout( bargap=0,title= "Stamper Run Mode")
                    st.write(fig_stamper_runmode)
                if option =="BOSCH1":
                    df_stamper_without_dash_for_machine_status = df_BOSCH1.iloc[1:480,:]
                    stamper_runmode = pd.to_numeric(df_stamper_without_dash_for_machine_status["BOSCH_1_Running         "])
                    stamper_runmode.replace(to_replace={1:0, 0:1}, inplace= True)


                    fig_stamper_runmode = go.Figure()

                    fig_stamper_runmode.add_trace(go.Bar(
                        x=df_stamper_without_dash_for_machine_status['DateAndTime            '],
                        y=stamper_runmode,
                        name='Primary Product',
                        marker_color='red',

                        ))
                    fig_stamper_runmode.update_layout( bargap=0,title= "BOSCH1 Run Mode")
                    st.write(fig_stamper_runmode)
                if option =="BOSCH2":
                    df_stamper_without_dash_for_machine_status = df_BOSCH2.iloc[1:480,:]
                    stamper_runmode = pd.to_numeric(df_stamper_without_dash_for_machine_status["BOSCH_2_Running         "])
                    stamper_runmode.replace(to_replace={1:0, 0:1}, inplace= True)


                    fig_stamper_runmode = go.Figure()

                    fig_stamper_runmode.add_trace(go.Bar(
                        x=df_stamper_without_dash_for_machine_status['DateAndTime            '],
                        y=stamper_runmode,
                        name='Primary Product',
                        marker_color='red',

                        ))
                    fig_stamper_runmode.update_layout( bargap=0,title= "BOSCH2 Run Mode")
                    st.write(fig_stamper_runmode)
                if option =="CPM1":
                    df_stamper_without_dash_for_machine_status = df_CPM1.iloc[1:480,:]
                    stamper_runmode = pd.to_numeric(df_stamper_without_dash_for_machine_status["CPM_1_Running           "])
                    stamper_runmode.replace(to_replace={1:0, 0:1}, inplace= True)


                    fig_stamper_runmode = go.Figure()

                    fig_stamper_runmode.add_trace(go.Bar(
                        x=df_stamper_without_dash_for_machine_status['DateAndTime            '],
                        y=stamper_runmode,
                        name='Primary Product',
                        marker_color='red',

                        ))
                    fig_stamper_runmode.update_layout( bargap=0,title= "CPM1 Run Mode")
                    st.write(fig_stamper_runmode)
                if option =="CPM2":
                    df_stamper_without_dash_for_machine_status = df_CPM2.iloc[1:480,:]
                    stamper_runmode = pd.to_numeric(df_stamper_without_dash_for_machine_status["CPM_1_Running           "])
                    stamper_runmode.replace(to_replace={1:0, 0:1}, inplace= True)


                    fig_stamper_runmode = go.Figure()

                    fig_stamper_runmode.add_trace(go.Bar(
                        x=df_stamper_without_dash_for_machine_status['DateAndTime            '],
                        y=stamper_runmode,
                        name='Primary Product',
                        marker_color='red',

                        ))
                    fig_stamper_runmode.update_layout( bargap=0,title= "CPM2 Run Mode")
                    st.write(fig_stamper_runmode)
 
                
        with fig_col3:
            #FBC = pd.read_csv("line.csv",3)
            #FBC.replace(np.nan,0)
            #FBC_without_dash= FBC.iloc[1:,:]                      
            #fig_fbc =  go.Figure(data=go.Bar(x=FBC_without_dash['DateAndTime            '],y=pd.to_numeric(df_CPM1_without_dash["TOTAL_FBC               "].iloc[1:481])))
            #fig_fbc.update_layout(
            #        title="FBC")
            #st.write(fig_fbc)
            
            
            ############## Plot for fault table
            def fault_time(file_name):
                    cpm1_fault = pd.read_csv(file_name)
                    top5_fault = cpm1_fault.iloc[1:6,3:4]
                    fault_freq = cpm1_fault.iloc[1:6,4:5]
                    figT= go.Figure(data= go.Table(header=dict(values=['Minor Stop Details', 'Duration (sec)']),cells=dict(values=[top5_fault,fault_freq])))
                    st.write(figT)
            if option == "CPM1":
                fault_time("CPM1faulttime2.csv")
            if option == "CPM2":
                fault_time("CPM2faulttime2.csv")   
            if option == "STAMPER":
                fault_time("stamperfaulttime.csv")    
            if option == "BOSCH1":
                fault_time("bosch1faulttime.csv")         
            if option == "BOSCH2":
                fault_time("bosch1faulttime.csv")       
                
          
        
        
        
        
            ## This is the table for the Downstream and upstream block
            if option =="BOSCH1":
                df_bosch_without_dash_for_block = df_BOSCH1.iloc[1:480,:]
                bosch1_upstream_block_count=df_bosch_without_dash_for_block['Bosch_01_Waiting_For_Upstream'].value_counts()[1]
                stop_type= ['Blocked by CPM 1', 'Waiting from Stamper']

                bosch1_downstream_block_count=df_bosch_without_dash_for_block['Bosch_01_Downstream_Not_Ready'].value_counts()[1]

                figT2= go.Figure(data= go.Table(header=dict(values=['', 'Block in minutes']),cells=dict(values=[stop_type,[bosch1_downstream_block_count,bosch1_upstream_block_count]])))

                st.write(figT2)
            
            
            
            
            if option=="STAMPER":
                
                df_bosch_without_dash_for_block = df_stamper.iloc[1:480,:]
                bosch1_upstream_block_count=df_bosch_without_dash_for_block['Stamper_Waiting_For_Upstream'].value_counts()[1]
                stop_type= ['Upstream waiting','A side blocked by BOSCH','B side blocked by BOSCH',]

                bosch1_downstream_block_count=df_bosch_without_dash_for_block['Stamper_Downstream_01_Stop'].value_counts()[1]
                
                bosch2_downstream_block_count=df_bosch_without_dash_for_block['Stamper_Downstream_02_Stop'].value_counts()[1]
                

                figT2= go.Figure(data= go.Table(header=dict(values=['', 'Block in minutes']),cells=dict(values=[stop_type,[bosch1_upstream_block_count,bosch1_downstream_block_count,bosch2_downstream_block_count]])))

                st.write(figT2)
                
                
                
                
            if option =="BOSCH2":
                
                df_bosch_without_dash_for_block = df_BOSCH2.iloc[1:480,:]
                bosch1_upstream_block_count=df_bosch_without_dash_for_block['BOSCH_2_Waiting_Upstream'].value_counts()[1]
                stop_type= ['Blocked by CPM 2', 'Waiting from Stamper']

                bosch1_downstream_block_count=df_bosch_without_dash_for_block['BOSCH_2_Downstream_Waiting'].value_counts()[1]

                figT2= go.Figure(data= go.Table(header=dict(values=['', 'Block in minutes']),cells=dict(values=[stop_type,[bosch1_downstream_block_count,bosch1_upstream_block_count]])))

                st.write(figT2)
            
              
            if option =="CPM1":
                
                
                df_bosch_without_dash_for_block2 = df_CPM1.iloc[1:480,:]
                bosch1_upstream_block_count2=479-(df_bosch_without_dash_for_block2['CPM_1_Under_Fault       ']==1).value_counts()[0]
                stop_type= ['Waiting for BOSCH 1']

                bosch1_downstream_block_count2=479-(df_bosch_without_dash_for_block2['CPM_1_Blocked_Downstream']==1).value_counts()[0]

                figT2= go.Figure(data= go.Table(header=dict(values=['', 'Block in minutes']),cells=dict(values=[stop_type,[bosch1_upstream_block_count2]])))

                st.write(figT2)
            if option == "CPM2":
                
                df_bosch_without_dash_for_block = df_CPM2.iloc[1:480,:]
                bosch1_upstream_block_count=479-(df_bosch_without_dash_for_block['CPM_1_Waiting_Upstream  ']).value_counts()[0]
                stop_type= ['Downstream Block','Waiting for BOSCH 2']

                bosch1_downstream_block_count=479-(df_bosch_without_dash_for_block['CPM_1_Blocked_Downstream']).value_counts()[0]

                figT2= go.Figure(data= go.Table(header=dict(values=['', 'Block in minutes']),cells=dict(values=[stop_type,[bosch1_downstream_block_count,bosch1_upstream_block_count]])))

                st.write(figT2)                
               
        
        time.sleep(1)
    #placeholder.empty()

2022-11-13 18:30:14.155 
  command:

    streamlit run C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0
77196.0


EmptyDataError: No columns to parse from file